In [44]:
%matplotlib inline
import numpy as np
import scipy as sp
import matplotlib as mpl
import matplotlib.cm as cm
import matplotlib.pyplot as plt
import pandas as pd
pd.set_option('display.width', 500)
pd.set_option('display.max_columns', 100)
pd.set_option('display.notebook_repr_html', True)
import seaborn as sns
sns.set_style("whitegrid")
sns.set_context("poster")
from sklearn.feature_extraction.text import CountVectorizer
import gensim
from sklearn.cross_validation import train_test_split
from collections import defaultdict
from pattern.en import parse
from pattern.en import pprint
from pattern.vector import stem, PORTER, LEMMA
from sklearn.feature_extraction import text 
stopwords=text.ENGLISH_STOP_WORDS
import re
punctuation = list('.,;:!?()[]{}`''\"@#$^&*+-|=~_')
DATAFILEPATH='/Users/Dilip_MBP/Documents/CS109/PROJECT/data/'


In [42]:
documents = ["Human machine interface for lab abc computer applications",
"A survey of user opinion of computer system response time",
            "The EPS user interface management system",
        "System and human system engineering testing of EPS",
           "Relation of user perceived response time to error measurement",
            "The generation of random binary unordered trees",
             "The intersection graph of paths in trees",
             "Graph minors IV Widths of trees and well quasi ordering",
             "Graph minors A survey"]

vectorizer_t = CountVectorizer(min_df=1, stop_words='english')
X_t = vectorizer_t.fit_transform(documents)
corpus_gensim_t = gensim.matutils.Sparse2Corpus(X_t, documents_columns=False)

In [9]:
bike_masterdf = pd.read_csv(DATAFILEPATH+'bike_details.csv')
print "Number of samples in the database:", bike_masterdf.shape[0]
bike_masterdf.head(2)


#tempdf = bike_masterdf.head(100).copy()
#tempdf.to_json('./temp_json.json')

Number of samples in the database: 59619


,Unnamed: 0,id,title,serial,manufacturer_name,frame_model,year,thumb,large_img,is_stock_img,stolen,stolen_location,date_stolen,registration_created_at,registration_updated_at,url,api_url,manufacturer_id,paint_description,name,frame_size,description,rear_tire_narrow,front_tire_narrow,type_of_cycle,test_bike,rear_wheel_size_iso_bsd,front_wheel_size_iso_bsd,handlebar_type_slug,frame_material_slug,front_gear_type_slug,rear_gear_type_slug,stolen_id,s_date_stolen,s_location,latitude,longitude,theft_description,locking_description,lock_defeat_description,police_report_number,police_report_department,s_rec_created_at,create_open311,sder_formatted_address,sder_street_number,sder_route,sder_postal_code,sder_neighborhood,sder_city,sder_county,sder_state,sder_country,date_stolen_epoch,registration_created_at_epoch,registration_updated_at_epoch,s_rec_created_at_epoch
0,0,50088,Schwinn Gateway,absent,Schwinn,Gateway,NaN,NaN,NaN,False,False,NaN,1970-01-01 00:00:00,2015-07-06 22:27:01,2015-11-03 05:36:54,https://bikeindex.org/bikes/50088,https://bikeindex.org/api/v1/bikes/50088,117,NaN,NaN,NaN,NaN,True,NaN,Bike,False,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1970-01-01 00:00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,1436221621,1446529014,0
1,1,60140,2016 Specialized Diverge Elite DSW,WSBC601010269K,Specialized,Diverge Elite DSW,2016,NaN,NaN,False,False,NaN,1970-01-01 00:00:00,2015-10-08 19:18:50,2015-11-03 05:34:30,https://bikeindex.org/bikes/60140,https://bikeindex.org/api/v1/bikes/60140,307,NaN,NaN,52cm,NaN,True,NaN,Bike,False,NaN,NaN,NaN,aluminum,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1970-01-01 00:00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,1444331930,1446528870,0


In [ ]:
#SPark setup

conf = (pyspark.SparkConf()
    .setMaster('local')
    .setAppName('pyspark')
    .set("spark.executor.memory", "4g"))
sc = pyspark.SparkContext(conf=conf)
sqlsc=SQLContext(sc)



In [8]:
thetext="The bike was locked to a sign pole by the corner of the superstore. The lock was a combination lock and i had \
tied chained my helmet to the bike. The bike was of make 'GT' with shimano gears and had a custom seat"
parse(thetext, tokenize=True, lemmata=True).split()

[[[u'The', u'DT', u'B-NP', u'O', u'the'],
  [u'bike', u'NN', u'I-NP', u'O', u'bike'],
  [u'was', u'VBD', u'B-VP', u'O', u'be'],
  [u'locked', u'VBN', u'I-VP', u'O', u'lock'],
  [u'to', u'TO', u'O', u'O', u'to'],
  [u'a', u'DT', u'B-NP', u'O', u'a'],
  [u'sign', u'NN', u'I-NP', u'O', u'sign'],
  [u'pole', u'NN', u'I-NP', u'O', u'pole'],
  [u'by', u'IN', u'B-PP', u'B-PNP', u'by'],
  [u'the', u'DT', u'B-NP', u'I-PNP', u'the'],
  [u'corner', u'NN', u'I-NP', u'I-PNP', u'corner'],
  [u'of', u'IN', u'B-PP', u'B-PNP', u'of'],
  [u'the', u'DT', u'B-NP', u'I-PNP', u'the'],
  [u'superstore', u'NN', u'I-NP', u'I-PNP', u'superstore'],
  [u'.', u'.', u'O', u'O', u'.']],
 [[u'The', u'DT', u'B-NP', u'O', u'the'],
  [u'lock', u'NN', u'I-NP', u'O', u'lock'],
  [u'was', u'VBD', u'B-VP', u'O', u'be'],
  [u'a', u'DT', u'B-NP', u'O', u'a'],
  [u'combination', u'NN', u'I-NP', u'O', u'combination'],
  [u'lock', u'NN', u'I-NP', u'O', u'lock'],
  [u'and', u'CC', u'O', u'O', u'and'],
  [u'i', u'NN', u'B-NP', u'O

In [10]:
num_topics=range(1,11)
num_topics

[1, 2, 3, 4, 5, 6, 7, 8, 9, 10]

In [6]:

a = np.arange(0,60,10)
a

array([ 0, 10, 20, 30, 40, 50])

In [2]:
import numpy as np
np.arange(1,11)

    
def get_lock_defeat(text) :
    
    try:
        text = re.sub(r'\.', '', text)
        
        lock_defeat = {'lock was cut, and left at the scene':'d1',
                       'lock was opened, and left unharmed at the scene': 'd2',
                       'lock is missing, along with the bike': 'd3',
                       'object that bike was locked to was broken,removed,or otherwise compromised':'d4',
                       'bike was not locked':'d5',
                       'other situation, please describe below':'d6'
                      }
        text = text.lower()
        
        return lock_defeat[text]
    except:
        return text

array([ 1,  2,  3,  4,  5,  6,  7,  8,  9, 10])

In [223]:

def prep_features(text, ref_dict):    
    try:
        text = re.sub(r'\.', '', text)
        text = text.lower()
        feature_list = np.zeros(len(ref_dict.keys()), dtype=np.int)
        pos = int(ref_dict[text])
        feature_list[pos] = 1
        #print type(feature_list)  
        return feature_list    
    except:
        nogo = np.repeat(np.nan, len(ref_dict.keys()))
        #print nogo
        return nogo
        
def get_lock_desc(dataframe) :
    lock_desc = {'u-lock':0,'two u-locks':1,'u-lock and cable':2,'chain with padlock':3,'cable lock':4,'heavy duty bicycle security chain':5,'not locked':6,'other':7}
    col_names=['lock_desc_1','lock_desc_2','lock_desc_3','lock_desc_4','lock_desc_5','lock_desc_6','lock_desc_7','lock_desc_8']
    
    try:
        
        desc_ser = dataframe.locking_description.apply(lambda x: prep_features(x, lock_desc))
        desc_list = desc_ser.tolist()
        #print desc_list
        desc_array = np.array(desc_list)
        #print "Array shape is: ", desc_array.shape
        desc_features = pd.DataFrame(desc_array,columns=col_names, index=dataframe.index)
        result = pd.concat([dataframe, desc_features], axis=1)
        
        return result
    except:
        #print "Boo"
        return dataframe


    
def get_lock_defeat(dataframe):
    lock_defeat = {'lock was cut, and left at the scene':0,
                     'lock was opened, and left unharmed at the scene': 1,
                       'lock is missing, along with the bike': 2,
                       'object that bike was locked to was broken,removed,or otherwise compromised':3,
                       'bike was not locked':4,
                       'other situation, please describe below':5
                      }
    colname_def_list = ['lock_defeat_1','lock_defeat_2','lock_defeat_3','lock_defeat_4','lock_defeat_5','lock_defeat_6']
    
    try:
        def_feature_ser = dataframe.lock_defeat_description.apply(lambda x: prep_features(x, lock_defeat))
        def_feature_list = def_feature_ser.tolist()
        def_feature_array = np.array(def_feature_list)
        def_features = pd.DataFrame(def_feature_array,columns=colname_def_list, index=dataframe.index)
        newdf = pd.concat([dataframe, def_features], axis=1)
        return newdf
    except:
        #print "Boo"
        return dataframe

In [222]:
tempdfa = bike_masterdf.head(200).copy()
print tempdfa.shape
abc = get_lock_desc(tempdfa)
print abc.shape
defg = get_lock_defeat(abc)
print defg.shape
abc.head(2)

(200, 57)
(200, 65)
(200, 71)


,Unnamed: 0,id,title,serial,manufacturer_name,frame_model,year,thumb,large_img,is_stock_img,stolen,stolen_location,date_stolen,registration_created_at,registration_updated_at,url,api_url,manufacturer_id,paint_description,name,frame_size,description,rear_tire_narrow,front_tire_narrow,type_of_cycle,test_bike,rear_wheel_size_iso_bsd,front_wheel_size_iso_bsd,handlebar_type_slug,frame_material_slug,front_gear_type_slug,rear_gear_type_slug,stolen_id,s_date_stolen,s_location,latitude,longitude,theft_description,locking_description,lock_defeat_description,police_report_number,police_report_department,s_rec_created_at,create_open311,sder_formatted_address,sder_street_number,sder_route,sder_postal_code,sder_neighborhood,sder_city,sder_county,sder_state,sder_country,date_stolen_epoch,registration_created_at_epoch,registration_updated_at_epoch,s_rec_created_at_epoch,lock_desc_1,lock_desc_2,lock_desc_3,lock_desc_4,lock_desc_5,lock_desc_6,lock_desc_7,lock_desc_8
0,0,50088,Schwinn Gateway,absent,Schwinn,Gateway,NaN,NaN,NaN,False,False,NaN,1970-01-01 00:00:00,2015-07-06 22:27:01,2015-11-03 05:36:54,https://bikeindex.org/bikes/50088,https://bikeindex.org/api/v1/bikes/50088,117,NaN,NaN,NaN,NaN,True,NaN,Bike,False,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1970-01-01 00:00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,1436221621,1446529014,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1,60140,2016 Specialized Diverge Elite DSW,WSBC601010269K,Specialized,Diverge Elite DSW,2016,NaN,NaN,False,False,NaN,1970-01-01 00:00:00,2015-10-08 19:18:50,2015-11-03 05:34:30,https://bikeindex.org/bikes/60140,https://bikeindex.org/api/v1/bikes/60140,307,NaN,NaN,52cm,NaN,True,NaN,Bike,False,NaN,NaN,NaN,aluminum,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1970-01-01 00:00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,1444331930,1446528870,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [163]:
lock_desc = {'u-lock':'0','two u-locks':'1','u-lock and cable':'2','chain with padlock':'3','cable lock':'4','heavy duty bicycle security chain':'5','not locked':'6','other':'7'}
print lock_desc['chain with padlock']
#print lock_desc.values()[2]

3


In [25]:
a = [1,6,7,9,88,45]
a[4] = 675
a[4]

675